Please hand in this completed notebook before next lab by uploading it to blackboard.

<font color="red"><em> Please double click on this cell, and replace its contents with your name </em></font>

Run the following cell by pressing shift-enter.  Answer the questions marked in <font color="red">red</font>.

In [ ]:
%pylab inline
from __future__ import division 

## Model

In lecture (and in your homework) you have modelled the Ammonia molecule by two amplitudes:  $\psi_L$ is the amplitude for the Nitrogen atom to be on the left, and $\psi_R$ is the amplitude for it to be on the right.  We argued by symmetry that

$$ i\partial_t \left(\begin{array}{c}\psi_L(t)\\\psi_R(t)\end{array}\right)=
\left(\begin{array}{cc}a&b\\ b&a\end{array}\right)\left(\begin{array}{c}\psi_L(t)\\\psi_R(t)\end{array}\right),$$

It turns out that $a$ is irrelevant to the dynamics.  We can get rid of it by taking $\psi\to e^{-i a t} \psi$.  Further, we can always choose our unit of time so that $b=-1$.  Thus in the first part of this notebook we will solve  

$$ i\partial_t \left(\begin{array}{c}\psi_L(t)\\\psi_R(t)\end{array}\right)=
\left(\begin{array}{cc}0&-1\\ -1&0\end{array}\right)\left(\begin{array}{c}\psi_L(t)\\\psi_R(t)\end{array}\right),$$

This is a simple enough equation that we can solve it analytically, but we want to develop numerical tools.  In particular, we will discretize time, and use a finite difference approximation.

To make the notation nice, I will write $$\vec\psi(t)=\left(\begin{array}{c}\psi_L(t)\\\psi_R(t)\end{array}\right),$$ and $$H=\left(\begin{array}{cc}0&-1\\ -1&0\end{array}\right)$$

In the last recitation you were introduced to finite differences in space.  Next recitation you will combine the two techniques, and look at finite differences in both space and time.

In the last two homeworks you saw a couple useful finite difference schemes.

$$ \mbox{ Euler:}\quad i\frac{\vec\psi(t+\delta t)-\vec \psi(t)}{\delta t}= H\vec\psi(t)$$

$$\mbox{ Backwards Euler:}\quad i\frac{\vec\psi(t+\delta t)-\vec \psi(t)}{\delta t}= H\vec\psi(t+\delta t)$$

You saw in your homework that neither of these conserve probability:  The normalization of $\vec \psi$ gets messed up.  The simplest finite difference approximation which does not mess up the normalization is the Unitary (or Semi-Implicit) Euler method:

$$\mbox{ Unitary Euler:}\quad i\frac{\vec\psi(t+\delta t)-\vec \psi(t)}{\delta t}= H\left(\frac{\vec\psi(t+\delta t)+\vec\psi(t)}{2}\right)$$

<font color="red">Rewrite this equation so that $\psi(t+\delta t)$ appears on the left, and $\psi(t)$ on the right.  Double click on this cell, and fill in the ??? in the equation 
$$\left(1+\frac{i H \delta t}{2}\right)\vec\psi(t+\delta t)= \left(???\right)\vec\psi(t)$$
</font>

By inverting the matrix $(1+i H \delta t/2)$, one can write $$ \vec\psi(t+\delta t)=U\vec\psi(t)$$ where $U$ is a $2\times 2$ matrix.  

We can write $U$ as the product of two matrices

$$U=\left(1+i H \delta t/2\right)^{-1} \left(1-i H \delta t/2\right)$$

<font color="red"> Edit the following equation, replacing $a,b,c,d$ with expressions involving $\delta t$, $i$, and numerical factors.  [Hint: you might want to first write the solution on a piece of paper, then type it in.  The act of typing can be disorienting, and it is better to separate figuring out the "minus" signs from figuring out how to get it to typeset right]
$$
(1+i H \delta t/2)=\left(\begin{array}{cc} a & b \\ c & d \end{array}\right)
$$
</font>

<font color="red"> Edit the following equation, replacing $e,f,g,h$ with expressions involving $\delta t$, $i$, and numerical factors.  [Again, you should probably write it on a piece of paper first.]
$$
(1-i H \delta t/2)=\left(\begin{array}{cc} e & f \\ g & h \end{array}\right)
$$
</font>

In Python one can build matrices with the following notation -- hit shift-enter on that cell

In [ ]:
U1= array([[1,2],[3,4]])

You can see the matrix by just typing the name of it and shift-enter

In [ ]:
U1

Better yet, you can load the showmat package

In [ ]:
from showmat import showmat
showmat(U1)

You can invert matrices using the following notation

In [ ]:
Uinv1=inv(U1)
showmat(Uinv1)

We can verify that this is indeed the inverse by taking the matrix product

In [ ]:
Uinv1.dot(U1)

In Python, you can define complex numbers using a lower case $j$, such as

In [ ]:
(3.+2.j)*(1-1j)

While there are ways to make Python work symbolically (like Mathematica) we will work only with numerical matrices in this part of the course.  Thus we need a function which generates $U$, given $\delta t$.

Replace the a,b,c,d,e,f,g,h in the following function

In [ ]:
def U(dt):
    """ U(dt) generates a 2x2 matrix which evolves the wavefunction for Ammonia by a time dt. 
    We use units where the level spacing is unity."""
    mat1=array([[a,b],[c,d]]) # this should be 1+i H dt/2
    mat2=array([[e,f],[g,h]]) # this should be 1-i H dt/2
    U=inv(mat1).dot(mat2)
    return U

Test it with the following code

In [ ]:
U2=U(0.1)
U2

<font color=red>Look up on Google how to take the determinent of U2.  Calculate the determinent. </font>

<font color="red">Is the matrix U Unitary, Hermitian, AntiHermitian, none of these?</font>  Write your answer here

## Time Evolution

To calculate the time evolution, we simply need to repeatedly multiply by this matrix.  For that we need "loops."  Here is a little "program" which calculates a time-sequence of wavefunctions.  [One could encapsulate this into a function, but I find many students are more comfortable simply copying and pasting code blocks like this one.]

In [ ]:
dt=0.1  # set timestep
evolve = U(dt) # generate matrix which evolves in time
psi0=array([1,0]) # initial condition
maxt= 10 # what number to integrate to

t=0
psi=psi0

tlist=[t]
psilist=[psi]

while (t<maxt):
    t=t+dt
    psi=evolve.dot(psi)
    tlist.append(t)
    psilist.append(psi)
    
psiarray=array(psilist) # convert to more convenient container

Here is an example of Plotting the probability of the Nitrogen being on the left

In [ ]:
plot(tlist,abs(psiarray[:,0])**2)
ylabel("$|\psi_L|^2$",fontsize=24)
xlabel("$t$",fontsize=24)
ylim(0,1)
title("Probability of being on left")

An important question is how small to take our step-size.  Modify the following code to produce a stepsize of dt=0.01.  

In [ ]:
dt=0.1  # set timestep -- you need to modify this line
evolve = U(dt) # generate matrix which evolves in time
psi0=array([1,0]) # initial condition
maxt= 10 # what number to integrate to

t=0
psi=psi0

tlist2=[t]
psilist2=[psi]

while (t<maxt):
    t=t+dt
    psi=evolve.dot(psi)
    tlist2.append(t)
    psilist2.append(psi)
    
psiarray2=array(psilist2) # convert to more convenient container

The following code makes a plot which compares the results for dt=0.1 and dt=0.01

In [ ]:
plot(tlist2,abs(psiarray2[:,0])**2,".")
plot(tlist,abs(psiarray[:,0])**2,"o")
ylim(0,1)
ylabel("$|\psi_L|^2$",fontsize=24)
xlabel("$t$",fontsize=24)
title("Probability of being on left")

Comment on the accuracy of the $dt=0.1$ result

<font color="red"> Give response here </font>

In the cells below, repeat the dt=0.1 calculation, but with the initial state being $|+\rangle=(1/\sqrt{2},1/\sqrt{2})$.  For obvious reasons, we call $|+\rangle$ a "stationary state".  <em> Note, your plot will look funny unless you include the command <tt>ylim(0,1)</tt> which sets the y-axis range.  The plotting functions sometimes have difficulty with automatically choosing the range </em>

## Stimulated Absorption and Emission

In the presence of a microwave field, the equations of motion become

$$ i\partial_t \left(\begin{array}{c}\psi_L(t)\\\psi_R(t)\end{array}\right)=
\left(\begin{array}{cc}E(t)&-1\\ -1&-E(t)\end{array}\right)\left(\begin{array}{c}\psi_L(t)\\\psi_R(t)\end{array}\right),$$

where $E(t)=\epsilon \cos(\omega t)$.

Now that $H$ is time dependent, the generalization of the Unitary Euler method is

$$i\frac{\vec\psi(t+\delta t)-\vec \psi(t)}{\delta t}= H(t+\delta t/2)\left(\frac{\vec\psi(t+\delta t)+\vec\psi(t)}{2}\right)$$

This can be rewritten as

$$\psi(t+\delta t)= U(t) \psi(t)$$

<font color="red"> Edit the following equation, replacing $a,b,c,d,e,f,g,h$ with expressions involving $\delta t$, $i$, $E$ (which is shorthand for $E(t+\delta t/2)$ and numerical factors.  [Again -- writing it out on paper first will make life easier.]
$$
U(t)=\left(\begin{array}{cc} a & b \\ c & d \end{array}\right)^{-1} \left(\begin{array}{cc} e & f \\ g & h \end{array}\right)
$$
</font>

Make a function which generates this $U$

In [ ]:
def drivenU(dt,E):
    """ U(dt,E) generates a 2x2 matrix which evolves the wavefunction for Ammonia by a time dt
    in the presence of an oscillating electric field.  The quantity E, should be the electric field
    at time t+dt/2.
    We use units where the level spacing is unity."""
    mat1=array([[a,b],[c,d]]) # this should be 1+i H dt/2
    mat2=array([[e,f],[g,h]]) # this should be 1-i H dt/2
    U=inv(mat1).dot(mat2)
    return U

Test your function.  At the very least make sure that it evalutes to a $2\times2$ matrix.  You can also verify that when $E=0$ it gives the same result as in the case without driving.

Write a little loop, which calculates the time evolution of the ammonia atom starting in the ground state $|+\rangle=(1/\sqrt{2},1/\sqrt{2})$.  Take $E=\epsilon \cos(\omega t)$ with $\epsilon=0.1$ and $\omega=1$.  This is off-resonant, and you should find very little stimulated absorption (at best a few percent).  You will have to generate $U$ inside your loop, as it is different at each time slice.  Use $dt=0.1$, and integrate to time $t=100$.  Plot the probability of being in the ground state $P_g=| \psi_L/\sqrt{2}+\psi_R/\sqrt{2}|^2$.  Label your graph.

Repeat with a resonant drive, where $\omega=2$.